In [1]:
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
import cv2
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from skimage.transform import resize

Using TensorFlow backend.
C:\anaconda3\envs\Entrenador\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\anaconda3\envs\Entrenador\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\anaconda3\envs\Entrenador\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\anaconda3\envs\Entrenador\lib\site-packages\tensorflow\python\fra

In [2]:
# Cargar el modelo entrenado y el indice de enfermedades segun el entrenamiento ***************+
path_model = 'diseases_Version2_BSC.h5py' # Nombre del modelo entrenado. Soporta cualquier formato que el keras permita
enfermedades = [
'Tomato___Bacterial_spot',
'Tomato___Early_blight',
'Tomato___healthy',
'Tomato___Septoria_leaf_spot'    
] # Todas las enfermedades con las que se entreno la red
# Cantidad total de enfermedades igual a numero de salidas de la ultima capa ***************
nClasses = len(enfermedades)

# Arquitectura de la red empleada para entrenar ***************
diseases_model = Sequential()
diseases_model = Sequential()
diseases_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(256,256,3)))
diseases_model.add(LeakyReLU(alpha=0.1))
diseases_model.add(MaxPooling2D((2, 2),padding='same'))
diseases_model.add(Dropout(0.5))

diseases_model.add(Flatten())
diseases_model.add(Dense(32, activation='linear'))
diseases_model.add(LeakyReLU(alpha=0.1))
diseases_model.add(Dropout(0.5))
diseases_model.add(Dense(nClasses, activation='softmax'))

diseases_model.load_weights(path_model) # Carga los pesos de la red entrenada (Literalmente cargar la red entrenada)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
# Configuracion de la cámara *****************************+
frameWidth= 720         # CAMERA RESOLUTION 720 pixeles ancho
frameHeight = 1080      # CAMERA RESOLUTION 1080 pixeles alto
brightness = 60         # Brillo un poco bajo solo porque estaba fuerte la luz. Ajustar segun sea conveniente

font = cv2.FONT_HERSHEY_SIMPLEX
############################################## 
# Set setup of the video camera
cap = cv2.VideoCapture('http://192.168.2.102:4747/video')  # Direccion de la camara web empleada por el celular
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, brightness)

False

In [21]:
# Codigo para vision artificial
threshold = 0.80        # PROBABLITY THRESHOLD
while True:
    
    # READ IMAGE
    images=[]
    success, imgOrignal = cap.read()
    # Show image Original
    cv2.imshow("Processed Image", imgOrignal)
    # PROCESS IMAGE (Aplicar las conversiones necesarias a la imagen para que esten en formato adecuado)
    image_resized = resize(imgOrignal, (256, 256),anti_aliasing=True,clip=False,preserve_range=True) # redimensionar a 256x256 RGB
    images.append(image_resized) # Agregar imagen al array para quedar con forma [1,256,256,3]
    images = np.array(images, dtype=np.uint8) #convierto de lista a numpy
    images = images.astype('float32') # Soporte a decimal
    images = images / 255.    # Normalizar (0-1)

    predicted_classes = diseases_model.predict(images) # Pos aplicar la prediccion
    lista  = predicted_classes.tolist() # Convertir a lista la prediccion para extraer sus caracteristicas
    probabilityValue = max(lista[0]) # Extrae el valor de probabilidad correspondiente a la enfermedad con mayor probabilidad de semejanza 
    classIndex = lista[0].index(probabilityValue) # Extraer el indice que corresponde a la enfermedad más probable

    if probabilityValue > threshold:
        #print(enfermedades[classIndex])
        cv2.putText(imgOrignal,str(classIndex)+" "+str(enfermedades[classIndex]), (120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(imgOrignal, str(round(probabilityValue*100,2) )+"%", (180, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.imshow("Result", imgOrignal)
   
    if cv2.waitKey(1) and 0xFF == ord('q'):
        break

KeyboardInterrupt: 

In [6]:
# Codigo para probar imagenes individuales **********+
images=[]
# AQUI ESPECIFICAMOS UNAS IMAGENES
filenames = ['testGaby/SeptorialLeafCamara.jpg']

for filepath in filenames:
    image = plt.imread(filepath,0)
    image_resized = resize(image, (256, 256),anti_aliasing=True,clip=False,preserve_range=True)
    images.append(image_resized)

X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
test_X = X.astype('float32')
test_X = test_X / 255.

predicted_classes = diseases_model.predict(test_X)

for i, img_tagged in enumerate(predicted_classes):
    print(img_tagged.tolist())
    print(filenames[i], enfermedades[img_tagged.tolist().index(max(img_tagged))])

[0.12356607615947723, 0.7377073168754578, 0.0002487436286173761, 0.13847781717777252]
testGaby/SeptorialLeafCamara.jpg Tomato___Early_blight
